In [1]:
import random
import nacl.encoding
import nacl.hash

from utils import *
from ore import *
from bplustree import *
from node import *
from read_data import *
from dtree import *
from mpc import sim_mpc_n
from he import sim_he_n
from rf import *

from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier

In [2]:
def test_ore_encoding_ore():
    ore = ORE(key='123')
    A = sorted([random.randint(-434531, 43454) for _ in range(100)])
    B = [scale_val(x) for x in A]
    C = [ore.encode(x) for x in B]
    co = Coordinator()
    co.receive_keys(C[:50], C[50:])
    co.mp_map.update(co.do_map)
    d = co.mp_map.copy()

    for i in range(len(C)):
        for j in range(i):
            x, y = OREncoding(d[C[j].x]), OREncoding(d[C[i].x])
            a = ore.compare(C[j], C[i])
            b = ore.compare(x, y)
            if a != -1 or a != b:
                print(i, j)

In [3]:
def simulate():
    co = Coordinator()
    do = DataOwner()
    mp = ModelProvider()
    mp.train_model()

    key_0 = str(random.randint(0, 1000))
    do.ore.key = key_0
    mp.ore.key = key_0
    do_encs = do.get_encodings()
    mp_encs = mp.get_encodings()
    co.receive_keys(mp_arr=mp_encs, do_arr=do_encs)

    mp.receive_map(co.mp_map)
    do.receive_map(co.do_map)

    enc_dataset = []
    for sample in do.dataset:
        _s = sample[:]
        for i in range(len(_s)-1):
            _v = do.ore.encode(scale_val(_s[i])).x
            _s[i] = OREncoding(do.ore_map[_v])
        enc_dataset.append(_s)

    ys = [x[-1] for x in enc_dataset]
    pred = mp.model.predict(enc_dataset)
    print(f'ACC: {accuracy_score(pred, ys) * 100:.2f}%')

# simulate()

In [4]:
import time


co = Coordinator()
do = DataOwner()
mp = ModelProvider()
mp.load_model('iris', 'dt')
# mp.train_model()

key_0 = str(random.randint(0, 1000))
do.ore.key = key_0
mp.ore.key = key_0
do_encs = do.get_encodings()
mp_encs = mp.get_encodings()

st = time.time()
co.receive_keys(mp_arr=mp_encs, do_arr=do_encs)

mp.receive_map(co.mp_map)
do.receive_map(co.do_map)

# do.dataset = read_rna()
# enc_dataset = []
# _ys = []
# _st = time.time()
# _i = 0
# for sample in do.dataset:
#     _s = sample[:]
#     _i += 1
#     for i in range(len(_s)-1):
#         _v = do.ore.encode(scale_val(_s[i])).x
#         _s[i] = OREncoding(do.ore_map[_v])
#     enc_dataset.append(_s)
#     if _i in _xs:
#         _ys.append((time.time()-_st)*1000)
#     if _i > _xs[-1]:
#         break
# ys = [x[-1] for x in enc_dataset]
# pred = mp.model.predict(enc_dataset)
# str(_ys)

  0%|          | 0/74 [00:00<?, ?it/s]

  0%|          | 0/82 [00:00<?, ?it/s]

All data inserted into tree.
All data mapped to encoding.


AttributeError: attribute 'threshold' of 'sklearn.tree._tree.Tree' objects is not writable

In [66]:
class TransDT:
    def __init__(self) -> None:
        pass

    def is_leaf(self, s):
        return self.children_left[s] == self.children_right[s]

    def predict_single(self, sample):
        now = 0
        while not self.is_leaf(now):
            tar = sample[self.feature[now]]
            if tar <= self.threshold[now]:
                now = self.children_left[now]
            else:
                now = self.children_right[now]
        return self.values[now]
    
    def predict(self, xs):
        return [self.predict_single(x) for x in xs]


def trans_sk_model(model):
    n_nodes = model.tree_.node_count
    children_left = model.tree_.children_left.tolist()
    children_right = model.tree_.children_right.tolist()
    feature = model.tree_.feature.tolist()
    threshold = model.tree_.threshold.tolist()
    values = model.tree_.value.tolist()

    classes = model.classes_
    def get_cls(s):
        idx = max(list(enumerate(s)), key=lambda x: x[1])[0]
        return classes[idx]
    values = [get_cls(s[0]) for s in values]

    ret = TransDT()
    ret.n_nodes = n_nodes
    ret.children_left = children_left
    ret.children_right = children_right
    ret.feature = feature
    ret.threshold = threshold
    ret.values = values

    return ret


In [45]:
md = pickle.load(open('model/iris.dt', 'rb'))

In [67]:
m = trans_sk_model(md)

In [37]:
ds = read_iris()
xs, ys = [x[:-1] for x in ds], [x[-1] for x in ds]

In [69]:
r1 = md.predict(xs).tolist()

In [70]:
r2 = m.predict(xs)